# 커스텀 데이터로 yolov5 모델 학습 후 tflite로 변환

## 1. 깃 클론 및 라이브러리 **설치**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

!pip install -qr requirements.txt
!pip install onnx_tf

## 2. roboflow에서 만든 커스텀 데이터 불러오기

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

%cd /content
!curl -L "https://app.roboflow.com/ds/HnYcCJveej?key=qQUiqRplDz" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## 3. 모델 세팅

In [ ]:
import yaml
from IPython.core.magic import register_line_cell_magic

with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])



@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## 4. 학습

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 32 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

## 5. 학습결과

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 텐서보드 안되면 이거 실행
from utils.plots import plot_results  
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## 6. best.pt -> best.tflite 변환

In [ ]:
!python /content/yolov5/export.py --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt --img 416 --batch 1

In [ ]:
# onnx to pb
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("/content/best.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("/content/output/best.pb")  # export the model

In [ ]:
import tensorflow as tf

saved_model_dir = '/content/output/best.pb'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open('./converted_model.tflite', 'wb').write(tflite_model)